In [ ]:
import glob
import os

import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

from implementation.training.utilities import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

In [ ]:
root_log_dirpath = r"C:\Documents\RoofSense\logs\3dgeoinfo"
ablation_log_dirpath = os.path.join(root_log_dirpath, "ablation")
optimization_log_dirpath = os.path.join(root_log_dirpath, "optimization")

study = optuna.load_study(
    # TODO: Change the study name to optimization.
    study_name="optim",
    storage=f"sqlite:///{optimization_log_dirpath}/storage.db",
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

In [ ]:
torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

# Last Step
for experiment_name, bands in zip(
    ["density", "lidar", "ndrm", "reflectance", "slope"],
    [
        [0, 1, 2, 3, 4, 5],  # density
        [0, 1, 2],  # lidar
        [0, 1, 2, 3, 4, 6],  # ndrm
        [0, 1, 2, 4, 5, 6],  # reflectance
        [0, 1, 2, 3, 5, 6],  # slope
    ],
):
    config = TrainingTaskHyperparameterTuningConfig(
        # Add constant parameters.
        # Encoder
        encoder="tu-resnet18d",
        zero_init_last=True,
        # Loss
        label_smoothing=0.1,
        # Optimizer
        optimizer="AdamW",
        # LR Scheduler
        lr_scheduler="CosineAnnealingLR",
        # Miscellaneous
        in_channels=len(bands),
        **best_params,
    )

    model = create_model(config)

    experiment_dirpath = os.path.join(ablation_log_dirpath, experiment_name)
    for ckpt_path in glob.glob(
        os.path.join(experiment_dirpath, "train", "version_*", "ckpts", "last.ckpt")
    ):
        print(
            f"Testing version: {[c for c in ckpt_path if c.isdigit()][0]!r} of experiment: {experiment_name!r}."
        )
        task: TrainingTask = TrainingTask.load_from_checkpoint(
            checkpoint_path=ckpt_path, model=model
        )

        datamodule = TrainingDataModule(
            root=r"C:\Documents\RoofSense\roofsense\dataset",
            bands=[
                # Align band numbers with those that rasterio expects.
                band + 1
                for band in bands
            ],
            append_lab=config.append_lab,
            append_tgi=config.append_tgi,
            slice=bands,
        )

        trainer = Trainer(
            logger=TensorBoardLogger(save_dir=experiment_dirpath, name="test"),
            benchmark=True,
        )
        trainer.test(task, datamodule=datamodule)